In [1]:
# Nov 20th, 2020
# I could not use dask here due to an error with metadata
# I'll move on for now but I could find the issue later

# February 12st, 2020
# This script compares classification in each classification system
# Second level matches specifications of Nemet and Johnson 2012

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import glob
import graphviz

file_list=glob.glob("data/citation/*")
dfs = [pd.read_parquet(f, columns=['patent_id']) for f in file_list]

In [ ]:
df=dfs[0]

In [ ]:
df.head()

In [ ]:
class_df=pd.read_parquet('data/wipo.parquet.gz')
df=df.merge(class_df, right_index=True, left_index=True)

In [ ]:
df=df.reset_index().set_index('patent_id') #reset_index avoid losing citation_id
df=df.merge(class_df, suffixes=['_citing', ''], right_index=True, left_index=True)

In [ ]:
df

In [ ]:
# class_df=dd.read_parquet('data/wipo.parquet.gz')

for i, df in enumerate(dfs):
#     df=element.compute()
#     class_df=class_df.compute()
    df=df.merge(class_df, right_index=True, left_index=True)
    df=df.reset_index().set_index('patent_id') #reset_index avoid losing citation_id
    df=df.merge(class_df, suffixes=['_citing', ''], right_index=True, left_index=True)
    df['external']=df['wipo_field_id_citing']==df['wipo_field_id']    
    df['far_external']=df['wipo_sector_id_citing']==df['wipo_sector_id']    
    if i==0:
        output=df.groupby(df.index).sum()
    else:
        output=pd.concat([output, df.groupby(df.index).sum()]) # following Nemet and Johson, this variable is relevant for citations made

In [ ]:
output.head()

In [ ]:
# df.isnull().sum()
# df.set_index('uuid', inplace=True)
output.to_parquet('data/int_ext_cit.parquet.gz', compression='gzip')